In [4]:
!pip freeze | grep token

asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1670263926556/work
conda-token @ file:///Users/paulyim/miniconda3/envs/c3i/conda-bld/conda-token_1662660369760/work
tokenizers==0.13.3


In [1]:
import sys
sys.path.append("/home/gridsan/vyuan/.local/lib/python3.9/site-packages")
sys.path.append("../src")

In [2]:
import os
import json
from pathlib import Path

import openai

from utils import data_utils

root = Path.cwd().parent.parent

In [3]:
dataset = "cifar10"
prompt_type = "important"

In [4]:
openai.api_key = open(os.path.join(os.path.expanduser("~"), ".openai_api_key"), "r").read()[:-1]

In [5]:
prompts = {
    "important" : "List the most important features for recognizing something as a \"goldfish\":\n\n-bright orange color\n-a small, round body\n-a long, flowing tail\n-a small mouth\n-orange fins\n\nList the most important features for recognizing something as a \"beerglass\":\n\n-a tall, cylindrical shape\n-clear or translucent color\n-opening at the top\n-a sturdy base\n-a handle\n\nList the most important features for recognizing something as a \"{}\":",
    "superclass" : "Give superclasses for the word \"tench\":\n\n-fish\n-vertebrate\n-animal\n\nGive superclasses for the word \"beer glass\":\n\n-glass\n-container\n-object\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around a \"tench\":\n\n- a pond\n-fish\n-a net\n-a rod\n-a reel\n-a hook\n-bait\n\nList the things most commonly seen around a \"beer glass\":\n\n- beer\n-a bar\n-a coaster\n-a napkin\n-a straw\n-a lime\n-a person\n\nList the things most commonly seen around a \"{}\":"
}

base_prompt = prompts[prompt_type]

In [6]:
Path.cwd()

PosixPath('/home/gridsan/vyuan/Label-free-CBM/label_free_cbm/notebooks')

In [7]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(root / cls_file, "r") as f:
    classes = f.read().split("\n")

In [12]:
feature_dict = {}

for i, label in enumerate(classes):
    feature_dict[label] = set()
    print("\n", i, label)
    for _ in range(2):
        response = openai.Completion.create(
              model="text-davinci-002",
              prompt=base_prompt.format(label),
              temperature=0.7,
              max_tokens=256,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
        #clean up responses
        features = response["choices"][0]["text"]
        features = features.split("\n-")
        features = [feat.replace("\n", "") for feat in features]
        features = [feat.strip() for feat in features]
        features = [feat for feat in features if len(feat)>0]
        features = set(features)
        feature_dict[label].update(features)
    feature_dict[label] = sorted(list(feature_dict[label]))


 0 airplane

 1 automobile

 2 bird

 3 cat

 4 deer

 5 dog

 6 frog

 7 horse

 8 ship

 9 truck


In [13]:
feature_dict

{'airplane': ['a large, metal body',
  'a tail',
  'a tail with a horizontal stabilizer',
  'engines',
  'engines attached to the wings',
  'landing gear',
  'two wings',
  'wings'],
 'automobile': ['a car-like body',
  'a metal body',
  'a steering wheel and dashboard inside',
  'a steering wheel in the front',
  'doors on the sides',
  'four metal plates that serve as wheels',
  'four rubber or plastic tires',
  'headlights and taillights',
  'windows all around'],
 'bird': ['a beak',
  'a tail',
  'ability to fly',
  'feathers',
  'legs',
  'two legs',
  'wings'],
 'cat': ['a long tail',
  'a small or medium-sized body',
  'a small to medium sized body',
  'four legs',
  'furry coat',
  'pointed ears',
  'whiskers'],
 'deer': ['a brown or reddish coat',
  'a compact body',
  'a long neck',
  'a long nose',
  'a tail',
  'a white tail',
  'antlers',
  'large antlers',
  'long legs',
  'long, slender legs'],
 'dog': ['a face with two eyes, a nose, and a mouth',
  'a four-legged animal

In [15]:
json_object = json.dumps(feature_dict, indent=4)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_{}_new.json".format(dataset, prompt_type), "w") as outfile:
    outfile.write(json_object)

In [16]:
json_object

'{\n    "airplane": [\n        "a large, metal body",\n        "a tail",\n        "a tail with a horizontal stabilizer",\n        "engines",\n        "engines attached to the wings",\n        "landing gear",\n        "two wings",\n        "wings"\n    ],\n    "automobile": [\n        "a car-like body",\n        "a metal body",\n        "a steering wheel and dashboard inside",\n        "a steering wheel in the front",\n        "doors on the sides",\n        "four metal plates that serve as wheels",\n        "four rubber or plastic tires",\n        "headlights and taillights",\n        "windows all around"\n    ],\n    "bird": [\n        "a beak",\n        "a tail",\n        "ability to fly",\n        "feathers",\n        "legs",\n        "two legs",\n        "wings"\n    ],\n    "cat": [\n        "a long tail",\n        "a small or medium-sized body",\n        "a small to medium sized body",\n        "four legs",\n        "furry coat",\n        "pointed ears",\n        "whiskers"\n    ]